<a href="https://colab.research.google.com/github/kfox-code/example-repo1/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install libraries, spaCy model and TextBlob corpora
!pip -q install spacy spacytextblob pandas numpy reportlab
!python -m spacy download en_core_web_md
!python -m textblob.download_corpora -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 22.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_da

In [2]:
# Imports and load spaCY pipeline
import pandas as pd
import numpy as np
import spacy

from spacytextblob.spacytextblob import SpacyTextBlob

# Load spaCy medium model (has vectors, useful for similarity)
nlp = spacy.load("en_core_web_md")

# Add TextBlob sentiment extension to spaCy pipeline
if "spacytextblob" not in nlp.pipe_names:
    nlp.add_pipe("spacytextblob")

nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'spacytextblob']

In [3]:
# Upload dataset CSV downloaded from Kaggle
from google.colab import files

uploaded = files.upload()
list(uploaded.keys())

Saving Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv to Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv


['Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv']

In [6]:
# Load CSV into a DataFrame
csv_filename = "Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv"

dataframe = pd.read_csv(csv_filename)

# Keep column names unchanged; select reviews.text
reviews_data = dataframe["reviews.text"]

# Remove rows where reviews.text is missing
clean_data = dataframe.dropna(subset=["reviews.text"]).copy()

print("Original rows:", len(dataframe))
print("Rows after dropna reviews.text:", len(clean_data))
clean_data.head(3)

Original rows: 28332
Rows after dropna reviews.text: 28332


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."


In [7]:
def preprocess_text(text: str) -> str:
    """
    Basic cleaning + stopword removal.
    - lower() standardises casing
    - strip() removes extra whitespace at ends
    - spaCy tokenisation to drop stopwords/punctuation/spaces
    """
    if not isinstance(text, str):
        return ""

    text = text.lower().strip()
    doc = nlp(text)

    tokens = []
    for token in doc:
        if token.is_space or token.is_punct:
            continue
        if token.is_stop:
            continue
        tokens.append(token.text)

    return " ".join(tokens)

# Apply preprocessing
clean_data["review_clean"] = clean_data["reviews.text"].astype(str).apply(preprocess_text)

clean_data[["reviews.text", "review_clean"]].head(3)


,reviews.text,review_clean
0,I order 3 of them and one of the item is bad q...,order 3 item bad quality missing backup spring...
1,Bulk is always the less expensive way to go fo...,bulk expensive way products like
2,Well they are not Duracell but for the price i...,duracell price happy


In [8]:
def predict_sentiment(review: str):
    """
    Returns (label, polarity, full_sentiment_tuple).
    TextBlob polarity is in [-1, 1].
    """
    doc = nlp(review)
    polarity = doc._.blob.polarity
    sentiment = doc._.blob.sentiment  # (polarity, subjectivity)

    if polarity > 0.05:
        label = "positive"
    elif polarity < -0.05:
        label = "negative"
    else:
        label = "neutral"

    return label, float(polarity), sentiment

# Quick sanity test
predict_sentiment("This product is amazing and I love it!")

('positive', 0.6125, Sentiment(polarity=0.6125, subjectivity=0.75))

In [9]:
# Test on sample reviews from dataset
sample_indices = [0, 1, 2, 10, 25]  # change if you want
for idx in sample_indices:
    review_raw = clean_data["reviews.text"].iloc[idx]
    review_clean = clean_data["review_clean"].iloc[idx]

    label, polarity, sent = predict_sentiment(review_clean)
    print("-" * 80)
    print("RAW:", review_raw)
    print("CLEAN:", review_clean)
    print(f"Predicted sentiment: {label} | polarity={polarity:.3f} | blob_sentiment={sent}")

--------------------------------------------------------------------------------
RAW: I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.
CLEAN: order 3 item bad quality missing backup spring pcs aluminum battery work
Predicted sentiment: negative | polarity=-0.450 | blob_sentiment=Sentiment(polarity=-0.44999999999999996, subjectivity=0.35833333333333334)
--------------------------------------------------------------------------------
RAW: Bulk is always the less expensive way to go for products like these
CLEAN: bulk expensive way products like
Predicted sentiment: negative | polarity=-0.500 | blob_sentiment=Sentiment(polarity=-0.5, subjectivity=0.7)
--------------------------------------------------------------------------------
RAW: Well they are not Duracell but for the price i am happy.
CLEAN: duracell price happy
Predicted sentiment: positive | polarity=0.800 | blob_sentiment=Sentiment(polarit

In [10]:
# Review similarity comparison
# Pick two reviews safely in-bounds
review_a = clean_data["reviews.text"].iloc[0]
review_b = clean_data["reviews.text"].iloc[1]

doc_a = nlp(review_a)
doc_b = nlp(review_b)

sim_score = doc_a.similarity(doc_b)
print("Review A:", review_a)
print("\nReview B:", review_b)
print("\nSimilarity score:", float(sim_score))

Review A: I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.

Review B: Bulk is always the less expensive way to go for products like these

Similarity score: 0.951943576335907
